<a href="https://colab.research.google.com/github/NazHub1993/Data_Analysis_Projects/blob/main/ML_Algorithms(Lasso_and_Ridge_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [374]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [375]:
df=pd.read_csv("Melbourne_housing.csv")
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [376]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [377]:
df.nunique()

,0
Suburb,351
Address,34009
Rooms,12
Type,3
Price,2871
Method,9
SellerG,388
Date,78
Distance,215
Postcode,211


In [378]:
df.isna().sum()

,0
Suburb,0
Address,0
Rooms,0
Type,0
Price,7610
Method,0
SellerG,0
Date,0
Distance,1
Postcode,1


#Taking only the columns that will come to use

In [379]:
cols_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount',
               'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Price']
df=df[cols_to_use]
df.head()


,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0,1465000.0


#Trying to handle the missing values

In [380]:
cols_to_fill_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']
df[cols_to_fill_zero]=df[cols_to_fill_zero].fillna(0)

df['Landsize']=df['Landsize'].fillna(df.Landsize.mean())
df['BuildingArea']=df['BuildingArea'].fillna(df.BuildingArea.mean())

In [381]:
df.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,0
Distance,0
CouncilArea,3
Bedroom2,0


#To dro the columns with missing values

In [382]:
df.dropna(inplace=True)

In [383]:
df.head()

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,160.2564,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0000,1035000.0
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0000,1465000.0
5,Abbotsford,3,h,PI,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,94.0,160.2564,850000.0
6,Abbotsford,4,h,VB,Nelson,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,1.0,2.0,120.0,142.0000,1600000.0


In [384]:
df.isna().sum()


,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,0
Propertycount,0
Distance,0
CouncilArea,0
Bedroom2,0


In [385]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop('Price',axis='columns'),df.Price,test_size=0.05)

In [386]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27244 entries, 1 to 34856
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         27244 non-null  object 
 1   Rooms          27244 non-null  int64  
 2   Type           27244 non-null  object 
 3   Method         27244 non-null  object 
 4   SellerG        27244 non-null  object 
 5   Regionname     27244 non-null  object 
 6   Propertycount  27244 non-null  float64
 7   Distance       27244 non-null  float64
 8   CouncilArea    27244 non-null  object 
 9   Bedroom2       27244 non-null  float64
 10  Bathroom       27244 non-null  float64
 11  Car            27244 non-null  float64
 12  Landsize       27244 non-null  float64
 13  BuildingArea   27244 non-null  float64
 14  Price          27244 non-null  float64
dtypes: float64(8), int64(1), object(6)
memory usage: 3.3+ MB


In [387]:
df.groupby('Suburb').describe

<bound method GroupBy.describe of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7b76496389b0>>

In [388]:
preprocessor=ColumnTransformer(
    transformers=[
        ('cat_encode',OneHotEncoder(drop='first',handle_unknown='ignore'),
         ['Suburb','Type','Method','SellerG','Regionname','CouncilArea'])
        ],
    remainder='passthrough'
)

from sklearn.pipeline import Pipeline
pipe=Pipeline(
    [
        ('preprocessor',preprocessor),
        ('model',LinearRegression())
    ]
)
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print(y_pred)












[ 944822.74116227 2454533.20164376 1379725.64387196 ... 1215586.36109663
  415074.31785775  481650.55872446]


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [389]:
y_test

,Price
28128,1050000.0
153,2100000.0
26512,1450000.0
252,766000.0
25887,512000.0
...,...
4137,396000.0
6351,1215000.0
7337,1102000.0
8685,550000.0


In [390]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test,y_pred))

190166677753.04532


In [391]:
r_score=pipe.score(x_test,y_test)
print(r_score)

0.6212054457740745


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [392]:
r_score=pipe.score(x_train,y_train)
print(r_score)

0.6694016988956797


# But still it does better on training data. I also want it to perform well on test_data.So I will use L1 and L2 regularization that would increase mean_square error if overfitting occurs.So by giving penalty to the model I am making it understand to not to do overfitting!!!!

In [393]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler


num_cols = x_train.select_dtypes(include=['int64', 'float64']).columns
preprocessor=ColumnTransformer(
    transformers=[
        ('cat_encode',OneHotEncoder(drop='first',handle_unknown='ignore'),
         ['Suburb','Type','Method','SellerG','Regionname','CouncilArea']),
        ('num_encode',StandardScaler(),num_cols)
        ],
    remainder='passthrough'
)

from sklearn.pipeline import Pipeline
pipe=Pipeline(
    [
        ('preprocessor',preprocessor),
        ('model',Lasso(alpha=50,max_iter=10000,tol=0.1))
    ]
)
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print(y_pred)
print(pipe.score(x_train,y_train))
print(pipe.score(x_test,y_test))

[ 941003.16429968 2417577.09904193 1405373.33594231 ... 1164560.95419749
  451275.69992509  438766.17516011]
0.6794100479356516
0.6247754893213302


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [394]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler



num_cols = x_train.select_dtypes(include=['int64', 'float64']).columns
preprocessor=ColumnTransformer(
    transformers=[
        ('cat_encode',OneHotEncoder(drop='first',handle_unknown='ignore'),
         ['Suburb','Type','Method','SellerG','Regionname','CouncilArea']),
        ('num_encode',StandardScaler(),num_cols)
        ],
    remainder='passthrough'
)

from sklearn.pipeline import Pipeline
pipe=Pipeline(
    [

        ('preprocessor',preprocessor),
        ('model',Ridge(alpha=50,max_iter=10000,tol=0.1))
    ]
)
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
print(y_pred)
print(pipe.score(x_train,y_train))
print(pipe.score(x_test,y_test))

[1054747.02473832 2072699.11586707 1263778.34683787 ... 1139651.35575792
  464999.71885162  191852.05108688]
0.61873000255032
0.5667610573859398


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


#So this was the implementation of Lasso and Ridge Regression that reduces overfitting by penalizing the model of increased mean_squared_error!!!